In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.7/350.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download(['punkt', 'averaged_perceptron_tagger', 'universal_tagset', 'maxent_ne_chunker','wordnet','stopwords','brown'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from functools import lru_cache

# Assuming legal-specific stopwords are added to the default English list
LEGAL_STOPWORDS = set(stopwords.words('english'))

# Precompile regular expressions for efficiency
PUNCT_RE = re.compile(r'[^\w\s]')  # You might want to adjust this to keep certain legal symbols
CITATION_RE = re.compile(r'\d+\s+[A-Z][a-z]+\.?\s+\d+')  # Example pattern for legal citations like '123 U.S. 456'

def preprocess_text(text):
    text = text.lower()

    # Remove legal citations (customize regex as needed)
    text = CITATION_RE.sub('', text)

    # Remove punctuation
    text = PUNCT_RE.sub('', text)

    # Tokenize text
    words = word_tokenize(text)

    # Lemmatize and remove stopwords
    lemmatizer = WordNetLemmatizer()
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in LEGAL_STOPWORDS]

    return ' '.join(processed_words)


**mvbhat/verdicts data**

In [ ]:

from datasets import load_dataset
from haystack import Document

# Load the dataset
dataset = load_dataset("mvbhat/verdicts", split="train")

# Print the first record to see the data structure
print(dataset[0])

# Create documents with separate content and optional metadata
docs = [Document(content=doc["facts"], meta={"verdict": doc["verdict"]}) for doc in dataset]


INFO:haystack.telemetry._telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


Generating train split:   0%|          | 0/1001 [00:00<?, ? examples/s]

{'facts': 'In 1970, Jane Roe (a fictional name used in court documents to protect the plaintiff’s identity) filed a lawsuit against Henry Wade, the district attorney of Dallas County, Texas, where she resided, challenging a Texas law making abortion illegal except by a doctor’s orders to save a woman’s life. In her lawsuit, Roe alleged that the state laws were unconstitutionally vague and abridged her right of personal privacy, protected by the First, Fourth, Fifth, Ninth, and Fourteenth Amendments.\n', 'verdict': "Inherent in the Due Process Clause of the Fourteenth Amendment is a fundamental “right to privacy” that protects a pregnant woman’s choice whether to have an abortion. However, this right is balanced against the government’s interests in protecting women's health and protecting “the potentiality of human life.” The Texas law challenged in this case violated this right.\nJustice Harry Blackmun delivered the opinion for the 7-2 majority of the Court.\nFirst, the Court consider

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [ ]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"], policy="skip")


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

1001

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [21]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


In [5]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-4-turbo")

INFO:haystack.telemetry._telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


Enter OpenAI API key:··········


In [ ]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [17]:
from sklearn.metrics import jaccard_score
from nltk.translate import bleu_score
from nltk.metrics.distance import edit_distance
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer  # Importing the missing class

def calculate_jaccard_similarity(answers):
    mlb = MultiLabelBinarizer()
    answer_tokens = [set(answer.split()) for answer in answers]
    encoded_answers = mlb.fit_transform(answer_tokens)
    return np.mean([jaccard_score(encoded_answers[i], encoded_answers[j]) for i in range(len(answers)) for j in range(i + 1, len(answers))])

def calculate_bleu_score(answers):
    reference = answers[0]  # Using the first answer as the reference
    return np.mean([bleu_score.sentence_bleu([reference.split()], a.split()) for a in answers[1:]])

def calculate_edit_distance(answers):
    return np.mean([edit_distance(answers[i], answers[j]) for i in range(len(answers)) for j in range(i + 1, len(answers))])

def calculate_cosine_similarity(answers):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(answers)
    similarity_matrix = cosine_similarity(embeddings)
    return (similarity_matrix.sum() - similarity_matrix.trace()) / (len(answers) * (len(answers) - 1))

def calculate_semantic_similarity(answers):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(answers)
    similarity_matrix = util.pytorch_cos_sim(embeddings, embeddings)
    return (similarity_matrix.sum().item() - similarity_matrix.trace().item()) / (len(answers) * (len(answers) - 1))


In [ ]:
questions = [
    "What was the impact of the Supreme Court's decision in Miranda v. Arizona on police interrogation practices?",
    "How has the Supreme Court interpreted the Second Amendment in recent decisions?",
    "What are the legal precedents set by Roe v. Wade regarding abortion rights?",
    "How do Supreme Court rulings affect federal immigration law?",
    "What role does the Supreme Court play in determining the constitutionality of a law?"
]


In [ ]:
# Initialize accumulators for each metric
total_jaccard = 0
total_bleu = 0
# total_edit_dist = 0
total_cosine_sim = 0
total_semantic_sim = 0
num_questions = len(questions)

# Calculating metrics for each question
for question in questions:
    answers = [basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})["llm"]["replies"][0] for _ in range(5)]

    jaccard = calculate_jaccard_similarity(answers)
    bleu = calculate_bleu_score(answers)
    # edit_dist = calculate_edit_distance(answers)
    cosine_sim = calculate_cosine_similarity(answers)
    semantic_sim = calculate_semantic_similarity(answers)

    # Accumulate metrics
    total_jaccard += jaccard
    total_bleu += bleu
    # total_edit_dist += edit_dist
    total_cosine_sim += cosine_sim
    total_semantic_sim += semantic_sim

    print(f"Question: {question}")
    print(f"Jaccard Index: {jaccard}")
    print(f"BLEU Score: {bleu}")
    # print(f"Edit Distance: {edit_dist}")
    print(f"Cosine Similarity: {cosine_sim}")
    print(f"Semantic Similarity: {semantic_sim}\n")

# Calculate averages
avg_jaccard = total_jaccard / num_questions
avg_bleu = total_bleu / num_questions
# avg_edit_dist = total_edit_dist / num_questions
avg_cosine_sim = total_cosine_sim / num_questions
avg_semantic_sim = total_semantic_sim / num_questions

# Print average results
print("Average Metrics Across All Questions:")
print(f"Average Jaccard Index: {avg_jaccard}")
print(f"Average BLEU Score: {avg_bleu}")
# print(f"Average Edit Distance: {avg_edit_dist}")
print(f"Average Cosine Similarity: {avg_cosine_sim}")
print(f"Average Semantic Similarity: {avg_semantic_sim}")


INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm


Question: What role does the Supreme Court play in determining the constitutionality of a law?
Jaccard Index: 0.2583320317565316
BLEU Score: 0.13846075667564056
Cosine Similarity: 0.8728572845458984
Semantic Similarity: 0.8728573560714722

Average Metrics Across All Questions:
Average Jaccard Index: 0.2977001111509098
Average BLEU Score: 0.19065445328845027
Average Cosine Similarity: 0.9078850746154785
Average Semantic Similarity: 0.9078851127624512


**U.S. states supreme court verdicts data**

In [5]:
from datasets import load_dataset
from haystack import Document

# Load the dataset
dataset = load_dataset("macadeliccc/US-LegalKit", split="train")

# Filter and create documents more safely
docs = []
for entry in dataset:
    year = entry.get('date', '')[-4:]
    if year.isdigit() and int(year) > 2018:
        content = entry.get("document", "")
        state = entry.get("state", "")
        date = entry.get("date", "")
        if content:  # Ensure there is content before creating the Document
            doc = Document(content=content, meta={"state": state, "date": date})
            docs.append(doc)


INFO:haystack.telemetry._telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [6]:
print(len(docs))

36024


In [6]:
def adaptive_chunk_document(doc, max_tokens=512):
    """
    Split a legal document into adaptive parts based on natural divisions such as sections or paragraphs,
    each part aiming to be under a maximum number of tokens.
    """
    paragraphs = doc.content.split('\n')  # Assuming paragraphs are separated by new lines
    current_chunk = []
    current_length = 0
    chunks = []

    for paragraph in paragraphs:
        paragraph_tokens = paragraph.split()
        paragraph_length = len(paragraph_tokens)

        # Check if adding this paragraph exceeds max tokens
        if current_length + paragraph_length > max_tokens:
            if current_chunk:
                # Finalize current chunk
                chunk_content = " ".join(current_chunk)
                chunk_meta = doc.meta.copy()
                chunk_meta['chunk_id'] = f"{doc.meta.get('id')}_part_{len(chunks) + 1}"
                chunks.append(Document(content=chunk_content, meta=chunk_meta))
            # Start a new chunk
            current_chunk = [paragraph]
            current_length = paragraph_length
        else:
            # Add paragraph to the current chunk
            current_chunk.append(paragraph)
            current_length += paragraph_length

    # Add the last chunk if there's remaining content
    if current_chunk:
        chunk_content = " ".join(current_chunk)
        chunk_meta = doc.meta.copy()
        chunk_meta['chunk_id'] = f"{doc.meta.get('id')}_part_{len(chunks) + 1}"
        chunks.append(Document(content=chunk_content, meta=chunk_meta))

    return chunks

# Apply adaptive chunking to each document in your dataset
docs_adaptive_split = []
for doc in docs:
    docs_adaptive_split.extend(adaptive_chunk_document(doc, max_tokens=512))


In [7]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [9]:
docs_with_embeddings = doc_embedder.run(docs_adaptive_split)
document_store.write_documents(docs_with_embeddings["documents"], policy="skip")


Batches:   0%|          | 0/3758 [00:00<?, ?it/s]

120246

In [10]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [12]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [13]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-4-turbo")

Enter OpenAI API key:··········


In [44]:
# Update the OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass("Enter new OpenAI API key:")

# Reinitialize the generator with the new API key
generator = OpenAIGenerator(model="gpt-4-turbo")


Enter new OpenAI API key:··········


In [14]:
from haystack import Pipeline

us_legal_kit_pipeline = Pipeline()
# Add components to your pipeline
us_legal_kit_pipeline.add_component("text_embedder", text_embedder)
us_legal_kit_pipeline.add_component("retriever", retriever)
us_legal_kit_pipeline.add_component("prompt_builder", prompt_builder)
us_legal_kit_pipeline.add_component("llm", generator)

# Now, connect the components to each other
us_legal_kit_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
us_legal_kit_pipeline.connect("retriever", "prompt_builder.documents")
us_legal_kit_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [15]:
questions = [
    "What have been the key changes in privacy laws in the United States since 2018?",
    "How has the interpretation of the First Amendment changed in the last decade?",
    "What are the implications of recent Supreme Court rulings on immigration?",
    "Can you detail significant cases regarding equal employment opportunities adjudicated since 2019?",
    "What legal precedents have been set regarding digital surveillance and individual privacy?"
]


In [18]:
# Initialize accumulators for each metric
total_jaccard = 0
total_bleu = 0
# total_edit_dist = 0
total_cosine_sim = 0
total_semantic_sim = 0
num_questions = len(questions)

# Calculating metrics for each question
for question in questions:
    answers = [us_legal_kit_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})["llm"]["replies"][0] for _ in range(5)]

    jaccard = calculate_jaccard_similarity(answers)
    bleu = calculate_bleu_score(answers)
    # edit_dist = calculate_edit_distance(answers)
    cosine_sim = calculate_cosine_similarity(answers)
    semantic_sim = calculate_semantic_similarity(answers)

    # Accumulate metrics
    total_jaccard += jaccard
    total_bleu += bleu
    # total_edit_dist += edit_dist
    total_cosine_sim += cosine_sim
    total_semantic_sim += semantic_sim

    print(f"Question: {question}")
    print(f"Jaccard Index: {jaccard}")
    print(f"BLEU Score: {bleu}")
    # print(f"Edit Distance: {edit_dist}")
    print(f"Cosine Similarity: {cosine_sim}")
    print(f"Semantic Similarity: {semantic_sim}\n")

# Calculate averages
avg_jaccard = total_jaccard / num_questions
avg_bleu = total_bleu / num_questions
# avg_edit_dist = total_edit_dist / num_questions
avg_cosine_sim = total_cosine_sim / num_questions
avg_semantic_sim = total_semantic_sim / num_questions

# Print average results
print("Average Metrics Across All Questions:")
print(f"Average Jaccard Index: {avg_jaccard}")
print(f"Average BLEU Score: {avg_bleu}")
# print(f"Average Edit Distance: {avg_edit_dist}")
print(f"Average Cosine Similarity: {avg_cosine_sim}")
print(f"Average Semantic Similarity: {avg_semantic_sim}")
#

INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm
INFO:haystack.core.pipeline.base:Warming up component text_embedder...
INFO:haystack.core.pipeline.pipeline:Running component text_embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:haystack.core.pipeline.pipeline:Running component retriever
INFO:haystack.core.pipeline.pipeline:Running component prompt_builder
INFO:haystack.core.pipeline.pipeline:Running component llm


Question: What legal precedents have been set regarding digital surveillance and individual privacy?
Jaccard Index: 0.2199819276153454
BLEU Score: 0.06201371966106016
Cosine Similarity: 0.8370719909667969
Semantic Similarity: 0.8370719909667969

Average Metrics Across All Questions:
Average Jaccard Index: 0.22509753830148474
Average BLEU Score: 0.07842074957711856
Average Cosine Similarity: 0.8069359588623047
Average Semantic Similarity: 0.8069359397888183


**verdicts by the Israeli Supreme Court in Hebrew data**

In [4]:
!pip install transformers datasets faiss-cpu sentence-transformers torch nltk rouge tqdm openai
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 78.3 MB/s eta 0:00:00


Option 1- Simulates answers from an expert and compares to the results of the model

In [6]:
import openai
import os
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from tqdm import tqdm
import numpy as np

# Ensure NLTK data is downloaded
import nltk
nltk.download(['punkt', 'averaged_perceptron_tagger', 'universal_tagset', 'maxent_ne_chunker','wordnet','stopwords','brown'])
key = 'api_key'
# Load the dataset
dataset = load_dataset("LevMuchnik/SupremeCourtOfIsrael", split='train')
client = openai.OpenAI(api_key=key)

def generate_answer(question, document_content):
    prompt = f"""
    Based on the following document, answer the question succinctly:

    Document: {document_content}

    Question: {question}
    Answer:
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant trained to provide concise legal advice."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return "An error occurred while generating the response."

def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return {
        'rouge-1': scores[0]['rouge-1']['f'],
        'rouge-2': scores[0]['rouge-2']['f'],
        'rouge-l': scores[0]['rouge-l']['f']
    }

def calculate_bleu_score(reference, candidate):
    return sentence_bleu([nltk.word_tokenize(reference)], nltk.word_tokenize(candidate))

def generate_reference_answer(question, document_content):
    prompt = f"""
    You are an expert in Israeli law. Based on the following document, provide a concise and accurate answer to the question:

    Document: {document_content}

    Question: {question}
    Expert Answer:
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an expert in Israeli law providing accurate and concise answers."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred generating reference answer: {e}")
        return "An error occurred while generating the reference answer."

def evaluate_model(num_samples=10):
    results = []

    for i in tqdm(range(num_samples)):
        document = dataset['text'][i]
        questions = [
            "What are the key legal issues presented in this case?",
            "Summarize the court's decision in this case.",
            "What precedents are cited in the judgment?"
        ]

        for question in questions:
            generated_answer = generate_answer(question, document)
            reference_answer = generate_reference_answer(question, document)

            rouge_scores = calculate_rouge_scores(reference_answer, generated_answer)
            bleu_score = calculate_bleu_score(reference_answer, generated_answer)

            results.append({
                'document_id': i,
                'question': question,
                'generated_answer': generated_answer,
                'reference_answer': reference_answer,
                'rouge_scores': rouge_scores,
                'bleu_score': bleu_score
            })

    return results

def print_evaluation_results(results):
    avg_rouge_1 = np.mean([r['rouge_scores']['rouge-1'] for r in results])
    avg_rouge_2 = np.mean([r['rouge_scores']['rouge-2'] for r in results])
    avg_rouge_l = np.mean([r['rouge_scores']['rouge-l'] for r in results])
    avg_bleu = np.mean([r['bleu_score'] for r in results])

    print(f"Average ROUGE-1 F1 Score: {avg_rouge_1:.4f}")
    print(f"Average ROUGE-2 F1 Score: {avg_rouge_2:.4f}")
    print(f"Average ROUGE-L F1 Score: {avg_rouge_l:.4f}")
    print(f"Average BLEU Score: {avg_bleu:.4f}")

    print("\nSample Evaluations:")
    for i, result in enumerate(results[:5]):  # Print first 5 results
        print(f"\nDocument {result['document_id']}, Question: {result['question']}")
        print(f"Generated Answer: {result['generated_answer']}")
        print(f"Reference Answer: {result['reference_answer']}")
        print(f"ROUGE-1 F1: {result['rouge_scores']['rouge-1']:.4f}")
        print(f"BLEU Score: {result['bleu_score']:.4f}")

# Run the evaluation
evaluation_results = evaluate_model(num_samples=10)
print_evaluation_results(evaluation_results)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
 10%|█         | 1/10 [00:40<06:03, 40.35s/it]/usr/local/lib/python3.10/

Average ROUGE-1 F1 Score: 0.4008
Average ROUGE-2 F1 Score: 0.2137
Average ROUGE-L F1 Score: 0.3772
Average BLEU Score: 0.0896

Sample Evaluations:

Document 0, Question: What are the key legal issues presented in this case?
Generated Answer: The key legal issues presented in this case involve a petition for the opening of a criminal investigation against a district attorney, based on allegations of deceit and fraud in legal proceedings related to the petitioner’s tax assessments. Additional issues pertain to the adequacy of the evidence presented by the petitioner to support his claims and the procedural propriety of refusing the criminal investigation request.
Reference Answer: The key legal issues presented in this case involve allegations of deceit and misconduct by a district attorney (respondent 2) from the Jerusalem District Prosecutor's Office. The petitioner, Yoram Yezdi, accuses the district attorney of misleading the court to obtain the cancellation of a decision that was fav

Option 2 - creates 5 answers for each question and compares the different results

In [5]:
import openai
import os
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from concurrent.futures import ThreadPoolExecutor, as_completed
nltk.download(['punkt', 'averaged_perceptron_tagger', 'universal_tagset', 'maxent_ne_chunker','wordnet','stopwords','brown'])

# nltk.download('punkt', quiet=True)
key = 'api_key'
# Load the dataset
dataset = load_dataset("LevMuchnik/SupremeCourtOfIsrael", split='train')
client = openai.OpenAI(api_key=key)

def generate_answer(question, document_content):
    prompt = f"""
    Based on the following document, answer the question succinctly and accurately:

    Document: {document_content[:1000]}  # Truncate to first 1000 characters

    Question: {question}
    Answer:
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are an assistant trained to provide concise and accurate legal advice based solely on the given document."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150  # Limit response length
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return "An error occurred while generating the response."

def calculate_cosine_similarities(answers):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(answers)
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Calculate average similarity for each answer
    avg_similarities = np.mean(cosine_sim, axis=1)

    # Calculate overall average similarity
    overall_avg_similarity = np.mean(avg_similarities)

    return avg_similarities, overall_avg_similarity

def process_sample(i):
    document = dataset['text'][i]
    questions = [
        "What are the key legal issues presented in this case?",
        "Summarize the court's decision in this case.",
        "What precedents are cited in the judgment?"
    ]

    results = []
    for question in questions:
        # Generate 5 answers for each question
        answers = [generate_answer(question, document) for _ in range(5)]

        # Calculate cosine similarities
        avg_similarities, overall_avg_similarity = calculate_cosine_similarities(answers)

        results.append({
            'document_id': i,
            'question': question,
            'answers': answers,
            'avg_similarities': avg_similarities,
            'overall_avg_similarity': overall_avg_similarity
        })

    return results

def evaluate_model(num_samples=10):
    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_sample, i) for i in range(num_samples)]
        for future in tqdm(as_completed(futures), total=num_samples):
            results.extend(future.result())
    return results

def print_evaluation_results(results):
    overall_avg_similarity = np.mean([r['overall_avg_similarity'] for r in results])

    print(f"Overall Average Similarity Across All Questions: {overall_avg_similarity:.4f}")

    print("\nSample Evaluations:")
    for i, result in enumerate(results[:5]):  # Print first 5 results
        print(f"\nDocument {result['document_id']}, Question: {result['question']}")
        print("Generated Answers:")
        for j, answer in enumerate(result['answers']):
            print(f"Answer {j+1}: {answer}")
            print(f"Average Similarity: {result['avg_similarities'][j]:.4f}")
        print(f"Overall Average Similarity: {result['overall_avg_similarity']:.4f}")

# Run the evaluation
evaluation_results = evaluate_model(num_samples=10)
print_evaluation_results(evaluation_results)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:

Generating train split:   0%|          | 0/751194 [00:00<?, ? examples/s]

100%|██████████| 10/10 [02:54<00:00, 17.47s/it]

Overall Average Similarity Across All Questions: 0.6521

Sample Evaluations:

Document 3, Question: What are the key legal issues presented in this case?
Generated Answers:
Answer 1: The key legal issue presented in this case involves a request for the addition of evidence by the petitioner, Yoram Yazdi.
Average Similarity: 0.6244
Answer 2: The key legal issue presented in this case is a petition filed by Yoram Yezdi requesting the addition of evidence in a proceeding before the Supreme Court of Israel.
Average Similarity: 0.5796
Answer 3: The key legal issue in this case involves a petition for the addition of evidence. The petitioner, Yoram Yazdi, is seeking to add evidence in a legal proceeding against the respondents, which include the State Prosecutor's Office represented by Edna Arbel and attorney Elad Rosenthal from the Jerusalem District Prosecutor's Office.
Average Similarity: 0.5406
Answer 4: The key legal issue presented in this document is the petitioner's request to add ev